In [ ]:
import json

# Results analysis 

## Disambiguation 

### Converting first output to JSONL

In [8]:
def parse_custom_json(file_path):
    with open(file_path, 'r') as f:
        content = f.read()

    # Fix it: wrap in list and separate blocks with commas
    fixed_content = "[" + content.replace("}{", "},{") + "]"

    return json.loads(fixed_content)

def write_jsonl_from_parsed_list(parsed_list, output_path):
    with open(output_path, 'w') as f:
        for item in parsed_list:
            json.dump(item, f)
            f.write("\n")

In [ ]:
result_file_path = './disambiguation_results.json' 

# Number of conflicts processed 
parsed_results = parse_custom_json(result_file_path)

# Write to clean JSONL format
write_jsonl_from_parsed_list(parsed_results, "disambiguation_results.jsonl")

### Analysis

In [ ]:
result_file_path = './disambiguation_results.jsonl' 

# this function is the code of the disambiguation code too
def load_solved_conflict_keys(jsonl_path):
    solved_keys = set()
    
    with open(jsonl_path, 'r') as f:
        for line in f:
            if line.strip():  # skip empty lines
                try:
                    entry = json.loads(line)
                    key = next(iter(entry))  # assuming one key per top-level dict
                    solved_keys.add(key)
                except Exception as e:
                    print(f"Error reading line: {line[:100]}...\n{e}")
    
    return solved_keys

print(f"Solved conflicts: {len(load_solved_conflict_keys('disambiguation_results.jsonl'))}")

Solved conflicts: 137
